In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
#read file 
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df = pd.read_csv("ex_submit.csv")

In [3]:
#drop Nan index and reset dataFrame's index
df_train = df_train.dropna()
df_train = df_train.reset_index(drop=True)

In [4]:
#read the len of this dataFrame
columns = df_train.columns
size_train = len(df_train[df_train.columns[0]])
size_test = len(df_test[df_test.columns[0]])

In [5]:
#set up storing data numpy array
train_examples = np.zeros((size_train, 22))
test_examples = np.zeros((size_test, 22))
train_labels = np.zeros((size_train,))
test_labels = np.ones((size_test,))

In [6]:
#transfrom date data
for i in range(size_train):
    s = df_train[columns[0]][i]
    date1 = datetime.datetime.strptime(s, "%Y-%m-%d")
    date2 = datetime.datetime.strptime(s.split("-")[0]+"-1-1", "%Y-%m-%d")
    train_examples[i][0] = (date1 - date2).days
    del date1
    del date2
    del s

for i in range(size_test):
    s = df_test[df_test.columns[0]][i]
    date1 = datetime.datetime.strptime(s, "%Y-%m-%d")
    date2 = datetime.datetime.strptime(s.split("-")[0]+"-1-1", "%Y-%m-%d")
    test_examples[i][0] = (date1 - date2).days
    del date1
    del date2
    del s

In [7]:
#put number data directly into dataFrame
n = 1
for col in range(1, 15):
    if col==7 or col==9:
        continue
    for i in range(size_train):
        train_examples[i][n] = df_train[columns[col]][i]
    n += 1

n = 1
for col in range(1, 15):
    if col==7 or col==9:
        continue
    for i in range(size_test):
        test_examples[i][n] = df_test[columns[col]][i]
    n += 1

In [8]:
#turn wind direction in to 4 columns
for col in [7, 9]:
    if col==7:
        j = 13
    elif col==9:
        j = 17
    dic = {"W":0,"E":1,"N":2,"S":3}
    for i in range(size_train):
        s = str(df_train[columns[col]][i])
        if not s or s=="" or s=='nan':
            s = "WENS"
        for c in s:
            train_examples[i][j+dic[c]] += 4/len(s)
    
    for i in range(size_test):
        s = str(df_test[columns[col]][i])
        if not s or s=="" or s=='nan':
            s = "WENS"
        for c in s:
            test_examples[i][j+dic[c]] += 4/len(s)

In [9]:
#transfrom non-number data into number data
for i in range(size_train):
    if df_train[columns[15]][i]=="Yes":
        train_examples[i][21] = 1
    elif df_train[columns[15]][i]=="No":
        train_examples[i][21] = 0

for i in range(size_test):
    if df_test[columns[15]][i]=="Yes":
        test_examples[i][21] = 1
    elif df_test[columns[15]][i]=="No":
        test_examples[i][21] = 0

for i in range(size_train):
    if df_train[columns[16]][i]=="Yes":
        train_labels[i] = 1
    elif df_train[columns[16]][i]=="No":
        train_labels[i] = 0

for i in range(size_train):
    if df_train[columns[16]][i]=="Yes":
        train_labels[i] = 1
    elif df_train[columns[16]][i]=="No":
        train_labels[i] = 0

In [10]:
#calculate the threshold about which data would rain next day
ratio = np.count_nonzero(train_labels == 1)/size_train
ratio

0.182548794489093

Keras

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

2022-11-17 14:49:39.811829: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [15]:
#set up keras model
X_train, y_train, X_test = np.delete(train_examples, [0], 1), to_categorical(train_labels), np.delete(test_examples, [0], 1)

keras_model = Sequential()
keras_model.add(Dense(256, activation='relu', input_dim=21))
keras_model.add(Dense(128, activation='relu'))
keras_model.add(Dense(64, activation='relu'))
keras_model.add(Dense(32, activation='relu'))
keras_model.add(Dense(16, activation='relu'))
keras_model.add(Dense(8, activation='relu'))
keras_model.add(Dense(4, activation='relu'))
keras_model.add(Dense(2, activation='softmax'))
keras_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.05), loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
#run keras model
keras_model.fit(x=X_train, y=y_train, batch_size = 128, epochs=1000, shuffle=True, verbose=1)
keras_prob = keras_model.predict(X_test, verbose=False)
keras_prediction = np.array([1 if i[1]>ratio else 0 for i in keras_prob])

Epoch 1/1000
55/55 [==============================] - 1s 2ms/step - loss: 27.6942 - accuracy: 0.6873
Epoch 2/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.4724 - accuracy: 0.8194
Epoch 3/1000
55/55 [==============================] - 0s 4ms/step - loss: 0.4707 - accuracy: 0.8206
Epoch 4/1000
55/55 [==============================] - 0s 3ms/step - loss: 0.4743 - accuracy: 0.8182
Epoch 5/1000
55/55 [==============================] - 0s 3ms/step - loss: 0.4763 - accuracy: 0.8169
Epoch 6/1000
55/55 [==============================] - 0s 4ms/step - loss: 0.4722 - accuracy: 0.8195
Epoch 7/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.4764 - accuracy: 0.8174
Epoch 8/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.4763 - accuracy: 0.8169
Epoch 9/1000
55/55 [==============================] - 0s 3ms/step - loss: 0.4685 - accuracy: 0.8226
Epoch 10/1000
55/55 [==============================] - 0s 3ms/step - loss: 0.4635 - accuracy: 0.825

In [ ]:
#store result to keras.csv
for i in range(size_test):
    df["ans"][i] = str(int(keras_prediction[i]))
    df["id"][i] = str(float(i))

df.to_csv('keras.csv', index=False)

/tmp/ipykernel_6505/1007258010.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ans"][i] = str(int(keras_prediction[i]))
/tmp/ipykernel_6505/1007258010.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id"][i] = str(float(i))


KNN

In [41]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn import tree
from sklearn.datasets import load_boston

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [42]:
#set up KNN model
X_train, y_train, X_test = np.delete(train_examples, [0], 1), train_labels, np.delete(test_examples, [0], 1)

KNN_model = KNeighborsClassifier(n_neighbors=50, weights="distance")
KNN_model.fit(X_train, y_train)
KNN_prob = KNN_model.predict_proba(X_test)
KNN_prediction = np.array([1 if i[1]>ratio else 0 for i in KNN_prob])

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f0df63c5280>
Traceback (most recent call last):
  File "/home/wei/anaconda3/envs/data/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/wei/anaconda3/envs/data/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/wei/anaconda3/envs/data/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/wei/anaconda3/envs/data/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [43]:
#save result to KNN.csv
for i in range(size_test):
    df["ans"][i] = str(int(KNN_prediction[i]))
    df["id"][i] = str(float(i))
df.to_csv('KNN.csv', index=False)

Xgboost

In [44]:
import xgboost as xgb
from rgf.sklearn import RGFClassifier

In [45]:
#set up xgb model
X_train, y_train, X_test = np.delete(train_examples, [0], 1), train_labels, np.delete(test_examples, [0], 1)
clf = xgb.XGBClassifier()
xgb_reg = xgb.XGBClassifier(learning_rate = 0.01, max_depth = 10, alpha = 12, n_estimators = 80)
xgb_reg.fit(X_train, y_train)
xgb_prob = xgb_reg.predict_proba(X_test)
xgb_prediction = np.array([1 if i[1]>ratio else 0 for i in xgb_prob])

/home/wei/anaconda3/envs/data/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:10:11] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [46]:
#save result to xgb.csv
for i in range(size_test):
    df["ans"][i] = str(int(xgb_prediction[i]))
    df["id"][i] = str(float(i))
df.to_csv('xgb.csv', index=False)

rgf

In [47]:
#set up rgf model
X_train, y_train, X_test = np.delete(train_examples, [0], 1), train_labels, np.delete(test_examples, [0], 1)

rgf = RGFClassifier(max_leaf=2000, algorithm="RGF_Sib", test_interval=300, loss="Log", verbose=False)
rgf.fit(X_train, y_train)
rgf_prob = rgf.predict_proba(X_test)
rgf_prediction = np.array([1 if i[1]>ratio else 0 for i in rgf_prob])

In [48]:
#save result to rgf.csv
for i in range(size_test):
    df["ans"][i] = str(int(rgf_prediction[i]))
    df["id"][i] = str(float(i))
df.to_csv('rgf.csv', index=False)

mix result

In [49]:
#set up every prediction weight and mix them according to it
prediction = [keras_prediction, KNN_prediction, xgb_prediction, rgf_prediction]
prob = [keras_prob, KNN_prob, xgb_prob, rgf_prob]
weight = [1, 5, 1, 5]

In [50]:
#prediction x weight to output outcome
pop_norm = np.array([0.0 for i in range(size_test)])
for i in range(4):
    for j in range(size_test):
        pop_norm[j] += prediction[i][j] * weight[i]
pop_norm = np.array([1 if pop_norm[i] > 4 else 0 for i in range(size_test)])

In [51]:
for i in range(size_test):
    df["ans"][i] = str(int(pop_norm[i]))
    df["id"][i] = str(float(i))
df.to_csv('mix.csv', index=False)